In [1]:
import random
import redis
import logging

In [2]:
random.seed(444)
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

In [3]:
logging.basicConfig()

In [4]:
class OutOfStockError(Exception):
    """Raised when PyHats.com is all out of today's hottest hat"""


def buyitem(r: redis.Redis, itemid: int) -> None:
    with r.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                # Get available inventory, watching for changes
                # related to this itemid before the transaction
                pipe.watch(itemid)
                nleft: bytes = r.hget(itemid, "quantity")
                if nleft > b"0":
                    pipe.multi()
                    pipe.hincrby(itemid, "quantity", -1)
                    pipe.hincrby(itemid, "npurchased", 1)
                    pipe.execute()
                    break
                else:
                    # Stop watching the itemid and raise to break out
                    pipe.unwatch()
                    raise OutOfStockError(f"Sorry, {itemid} is out of stock!")
            except redis.WatchError:
                # Stop watching the itemid and raise to break out
                error_count += 1
                logging.warning("WatchError #%d: %s; retrying", error_count, itemid)

    return None

In [29]:
r = redis.Redis(host='127.0.0.1', port='6379', db=1)

In [6]:
r

Redis<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=1>>>

In [7]:
r.keys()

[b'hat:1236154736', b'hat:56854717', b'hat:1326692461']

In [8]:
buyitem(r, "hat:56854717")

In [9]:
r.hmget("hat:56854717", "quantity", "npurchased")

[b'198', b'1']

In [10]:
from datetime import timedelta

In [11]:
r.setex("runner", timedelta(minutes=1), value="now you see me, now you don't")

True

In [24]:
r.ttl("runner")

-2

In [26]:
r.get("runner")

In [27]:
import datetime
import ipaddress

import redis


# where we put all the bad egg IP addresses
blacklist = set()
MAXVISITS = 15


ipwatcher = redis.Redis(host='127.0.0.1', port='6379', db=5)


while True:
    _, addr = ipwatcher.blpop("ips")
    addr = ipaddress.ip_address(addr.decode("utf-8"))
    now = datetime.datetime.utcnow()
    addrts = f"{addr}:{now.minute}"
    n = ipwatcher.incrby(addrts, 1)
    if n >= MAXVISITS:
        print(f"Hat bot detected!: {addr}")
        blacklist.add(addr)
    else:
        print(f"{now}: saw {addr}")
        _ = ipwatcher.expire(addrts, 60)

KeyboardInterrupt: 

In [28]:
r.lastsave()

datetime.datetime(2020, 2, 3, 12, 57, 16)

In [30]:
restaurant_484272 = {
    "name": "Ravagh",
    "type": "Persian",
    "address": {
        "street": {
            "line1": "11 E 30th St",
            "line2": "APT 1",
        },
        "city": "New York",
        "state": "NY",
        "zip": 10016
    }
}

In [31]:
import json

In [32]:
r.set(484272, json.dumps(restaurant_484272))

True

In [33]:
r.get(484272)

b'{"name": "Ravagh", "type": "Persian", "address": {"street": {"line1": "11 E 30th St", "line2": "APT 1"}, "city": "New York", "state": "NY", "zip": 10016}}'

In [34]:
from pprint import pprint

In [35]:
pprint(json.loads(r.get(484272)))

{'address': {'city': 'New York',
             'state': 'NY',
             'street': {'line1': '11 E 30th St', 'line2': 'APT 1'},
             'zip': 10016},
 'name': 'Ravagh',
 'type': 'Persian'}


In [36]:
from collections.abc import MutableMapping

In [ ]:
def setflat_skeys(r: redis.Redis, obj: dict, prefix: str, delim: str=":", *, _autopfix="") -> None:
    """Flatten 'obj' and set resulting field-value pairs into 'r'
    Calls `.set()` to write Redis instance inplace and returns None.
    
    `prefix` is an optional str that prefixes all keys.
    `delim` is the delimiter the separates the joined, flattened keys.
    `_autopfix` s used in recursive calls to create de-nested keys.
    
    The deepest-nested keys must be str, bytes, float, or int.
    Otherwise TypeError is raised.
    """
    allowed_vtypes = (str, bytes, float, int)
    for key, value = obj.items():
        key = _autopfix
    
    
    
    
    
    
    